### Read the dataset

In [3]:
!wget -O ml-latest-small.zip -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

for details.

--2025-01-27 12:31:21--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.52MB/s    in 0.6s    

2025-01-27 12:31:22 (1.52 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [4]:
# Load ther ratings.csv file into a pandas dataframe
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
ratings.shape

(100836, 4)

### Sparse Representation
* Since the data is sparse (most users have not rated most movies), we should represent the data in a sparse format to save memory and improve computational efficiency.
* The sparse format typically represents the data as a user-item matrix, where rows correspond to users, columns correspond to movies, and the values are the ratings.

### Features
* UserID and MovieID are categorical features and should be encoded using one-hot encoding or LabelEncoding
* Rating is the target variable (output) we want to predict
* Timestamp can be dropped as it won't contain any information regarding the ratings

In [7]:
# Label encode the user and movie ids
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoders
user_enc = LabelEncoder()
movie_enc = LabelEncoder()

# Fit and transform the training data
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
ratings['movie'] = movie_enc.fit_transform(ratings['movieId'].values)

# Display the first few rows of the test data
print("\nRatings data:")
print(ratings.head())



Ratings data:
   userId  movieId  rating  timestamp  user  movie
0       1        1     4.0  964982703     0      0
1       1        3     4.0  964981247     0      2
2       1        6     4.0  964982224     0      5
3       1       47     5.0  964983815     0     43
4       1       50     5.0  964982931     0     46


In [8]:
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

#create a sparse matrix of train data
n_users = ratings['user'].max() + 1
n_movies = ratings['movie'].max() + 1
print(n_users, n_movies)

sparse_matrix_train = csr_matrix((ratings['rating'], (ratings['user'], ratings['movie'])), shape=(n_users, n_movies))

print(sparse_matrix_train.shape)
print(sparse_matrix_train)

610 9724
(610, 9724)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100836 stored elements and shape (610, 9724)>
  Coords	Values
  (0, 0)	4.0
  (0, 2)	4.0
  (0, 5)	4.0
  (0, 43)	5.0
  (0, 46)	5.0
  (0, 62)	3.0
  (0, 89)	5.0
  (0, 97)	4.0
  (0, 124)	5.0
  (0, 130)	5.0
  (0, 136)	5.0
  (0, 184)	5.0
  (0, 190)	3.0
  (0, 197)	5.0
  (0, 201)	4.0
  (0, 224)	5.0
  (0, 257)	3.0
  (0, 275)	3.0
  (0, 291)	5.0
  (0, 307)	4.0
  (0, 314)	4.0
  (0, 320)	5.0
  (0, 325)	4.0
  (0, 367)	3.0
  (0, 384)	4.0
  :	:
  (609, 9238)	5.0
  (609, 9246)	4.5
  (609, 9256)	4.0
  (609, 9268)	5.0
  (609, 9274)	3.5
  (609, 9279)	3.5
  (609, 9282)	3.0
  (609, 9288)	3.0
  (609, 9304)	3.0
  (609, 9307)	2.5
  (609, 9312)	4.5
  (609, 9317)	3.0
  (609, 9324)	3.0
  (609, 9339)	4.0
  (609, 9341)	4.0
  (609, 9348)	3.5
  (609, 9371)	3.5
  (609, 9372)	3.5
  (609, 9374)	5.0
  (609, 9415)	4.0
  (609, 9416)	4.0
  (609, 9443)	5.0
  (609, 9444)	5.0
  (609, 9445)	5.0
  (609, 9485)	3.0


### Rank of the users x movie sparse matrix

In [9]:
import numpy as np
from scipy.sparse.linalg import svds

def rank_calculator(sparse_matrix_train, energy):
  
  #Convert to array
  train_data_matrix = sparse_matrix_train.toarray()

  #Perform SVD
  u, s, vt = svds(train_data_matrix, k=min(train_data_matrix.shape) - 1)

  #Sort the s values in descending order
  s = np.sort(s)[::-1]

  #Calculate the total energy
  total_energy = np.sum(s**2)
  current_energy = 0
  rank = 0
  for i in range(len(s)):
    current_energy += s[i]**2
    if current_energy/total_energy >= energy:
      rank = i
      break
    
  print("Rank of the matrix with {energy} energy: ", rank)
  return rank

### Create the error calculation function

In [10]:
from scipy.sparse import issparse

# error calculation function for sparse matrix

def error_calc(X_S, Y_S):

    # Convert sparse matrices to dense arrays if necessary
    if issparse(X_S):
        X_S = X_S.toarray()
    if issparse(Y_S):
        Y_S = Y_S.toarray()
    
    #Norm calculation X_S - Y_S
    normXminusY = np.linalg.norm(X_S - Y_S, ord=2)

    #Norm of Y_S
    norm_Y = np.linalg.norm(Y_S, ord=2)

    if norm_Y == 0:
        raise ValueError("Norm of Y is 0")

    #Calculate the error
    error = normXminusY/norm_Y
  
    return error

## Singular Value Projection Algorithm

SVP aims to find a low-rank matrix $X$ taht approximates an observed matrix $Y$ by solving:
$$
\min_{X} \|X - Y\|_F^2 \quad \text{subject to} \quad \text{rank}(X) \leq r
$$
where the rank $r$ is a fixed desired rank.

In [11]:
# Get the rank for 90% energy
rank = rank_calculator(sparse_matrix_train, 0.9)

Rank of the matrix with {energy} energy:  239


In [12]:
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import svds
from sklearn.base import BaseEstimator

In [13]:
#SVP Estimator class to use with cross validation in scikit learn

class SVPEstimator(BaseEstimator):
    def __init__(self, rank=10, max_iter=100, tol=1e-3, learning_rate=1):
        self.rank = rank
        self.max_iter = max_iter
        self.tol = tol
        self.learning_rate = learning_rate
        self.X = None
        self.fit_time = None
        self.error = None
        self.iter = None
        

    def fit(self, Y, y=None):
        start_time = time.time()
        #Init X
        self.X = lil_matrix(Y.shape, dtype = np.float64) #Sparse matrix with 64 bit float values for better stability
        iter = 0
        error = self.tol + 99
        prev_error = 100
        while(error > self.tol and iter < self.max_iter and prev_error > error):
            #Update the previous error
            prev_error = error

            #Update the iteration count
            iter += 1

            #Update the X matrix
            X_half = self.X + self.learning_rate * (Y - self.X)
            U, S, Vt = svds(X_half, k = self.rank)
            U_sparse = csr_matrix(U)
            Vt_sparse = csr_matrix(Vt)
            S_sparse = csr_matrix(np.diag(S))
            self.X = U_sparse @ S_sparse @ Vt_sparse 
            
            #Calculate the error
            error = self._error_calc(self.X, Y)

        self.error = error
        self.iter = iter
        self.fit_time = time.time() - start_time
        print(f"Fit time: {self.fit_time:.6f} seconds with {self.iter} iterations and error {self.error:.6f}.")
        return self
    
    def score(self, Y, y=None):
        # Extract observed entries in the validation fold
        val_indices = list(zip(Y.nonzero()[0], Y.nonzero()[1]))
        val_values = Y.data

        # Compute the error only on the observed entries in the validation fold
        error = self._error_calc_validation(self.X, val_indices, val_values)
        return -error  # For scikit-learn, higher score is better

    def _error_calc(self, X_S, Y_S):

        # Convert sparse matrices to dense arrays if necessary
        if issparse(X_S):
            X_S = X_S.toarray()
        if issparse(Y_S):
            Y_S = Y_S.toarray()
        
        #Norm calculation X_S - Y_S
        normXminusY = np.linalg.norm(X_S - Y_S, ord=2)

        #Norm of Y_S
        norm_Y = np.linalg.norm(Y_S, ord=2)

        if norm_Y == 0:
            raise ValueError("Norm of Y is 0")

        #Calculate the error
        error = normXminusY/norm_Y
    
        return error

    def _error_calc_validation(self, X, val_indices, val_values):
        
        # Extract predicted values at the validation indices
        pred_values = np.array([X[i, j] for (i, j) in val_indices])

        # Compute the Frobenius norm of the difference
        numerator = np.linalg.norm(pred_values - val_values, ord=2)

        # Compute the Frobenius norm of the validation values
        denominator = np.linalg.norm(val_values, ord=2)

        if denominator == 0:
            raise ValueError("Norm of validation values is 0")

        # Compute the relative error
        error = numerator / denominator

        return error


In [14]:
from sklearn.model_selection import cross_val_score
#Initialize the SVP Estimator
svp_estimator = SVPEstimator(rank=rank, max_iter=100, tol=1e-3, learning_rate=1)

#fit the model once for timing purposes
svp_estimator.fit(sparse_matrix_train)
SVPTimingOneIter = svp_estimator.fit_time/svp_estimator.iter
print("Fit time SVP One iteration: ", SVPTimingOneIter)

#Perform 5 fold CV
cv_scores = cross_val_score(svp_estimator, sparse_matrix_train, cv=5)

print("Cross validation errors: ", -cv_scores)
print("Mean CV error: ", -np.mean(cv_scores))

Fit time: 12.786731 seconds with 2 iterations and error 0.058972.
Fit time SVP One iteration:  6.393365502357483
Fit time: 9.587636 seconds with 2 iterations and error 0.052819.
Fit time: 14.999391 seconds with 3 iterations and error 0.054237.
Fit time: 14.557200 seconds with 3 iterations and error 0.055145.
Fit time: 9.556632 seconds with 2 iterations and error 0.057840.
Fit time: 9.384391 seconds with 2 iterations and error 0.055710.
Cross validation errors:  [0.95114829 0.9419772  0.95355127 0.95305303 0.95217684]
Mean CV error:  0.950381327654785


### Improve SVP
* Lets try to improve the algorithm by specifying a learning rate = 0.1

In [16]:
#Initialize the SVP Estimator
svp_estimator = SVPEstimator(rank=rank, max_iter=100, tol=1e-3, learning_rate=0.1)

#Perform 5 fold CV
cv_scores = cross_val_score(svp_estimator, sparse_matrix_train, cv=5, n_jobs=10)

print("Cross validation errors: ", -cv_scores)
print("Mean CV error: ", -np.mean(cv_scores))

Fit time: 462.462032 seconds with 29 iterations and error 0.055710.
Fit time: 475.604428 seconds with 30 iterations and error 0.057840.
Fit time: 486.367984 seconds with 29 iterations and error 0.055145.
Fit time: 500.586351 seconds with 31 iterations and error 0.052819.
Fit time: 500.770495 seconds with 30 iterations and error 0.054237.
Cross validation errors:  [0.95124582 0.94207106 0.9536366  0.95300789 0.95199927]
Mean CV error:  0.9503921279103466


### Conclusion SVP
* The SVP algorithm converges quite fast since the rank is fixed. (Hard rank constraint)
* There seems to be over-fitting happening looking at the cross validation error scores. This means that it is capturing the noise in the training folds. Probably the rank of the matrix is chosen to be too high.
* This SVP method does not have regularization. Maybe with regularization the results could be better, as we can penelize large values. (As we will see in the next algorithm)
* Increasing the data could also yield better results.
*  By adding the learning rate parameter of 0.1 the results are identical, which is expected, since in all cases the maximum number of iterations is not acheived, and we are at the same minimum point on the convex curve.
* The SVP algorithm is best suited for data that has less noise.

## Singular Value Threshold

SVT introduces a soft thresholding of the singular values which brings about a nuclear norm regularization.
$$
\min_{X} \|X - Y\|_F^2 + \lambda \|X\|_*
$$

In [17]:
#SVP Estimator class to use with cross validation in scikit learn

class SVTEstimator(BaseEstimator):
    def __init__(self, rank=10, max_iter=100, tol=1e-3, learning_rate=1, lambda_=0):
        self.rank = rank
        self.max_iter = max_iter
        self.tol = tol
        self.learning_rate = learning_rate
        self.lambda_ = lambda_
        self.X = None
        self.fit_time = None
        self.error = None
        self.iter = None
        

    def fit(self, Y, y=None):
        start_time = time.time()
        #Init X
        self.X = lil_matrix(Y.shape, dtype = np.float64) #Sparse matrix with 64 bit float values for better stability
        iter = 0
        error = self.tol + 99
        prev_error = 100
        while(error > self.tol and iter < self.max_iter and prev_error > error):
            #Update the previous error
            prev_error = error

            #Update the iteration count
            iter += 1

            #Update the X matrix
            X_half = self.X + self.learning_rate * (Y - self.X)
            U, S, Vt = svds(X_half, k = min(X_half.shape) - 1) # Take full possible rank
            S = np.maximum(S - self.lambda_, 0) #Soft thresholding
            U_sparse = csr_matrix(U)
            Vt_sparse = csr_matrix(Vt)
            S_sparse = csr_matrix(np.diag(S))
            self.X = U_sparse @ S_sparse @ Vt_sparse 

            #Calculate the error
            error = self._error_calc(self.X, Y)

        self.error = error
        self.iter = iter
        self.fit_time = time.time() - start_time
        print(f"Fit time: {self.fit_time:.6f} seconds with {self.iter} iterations and error {self.error:.6f}.")
        return self
    
    def score(self, Y, y=None):
        # Extract observed entries in the validation fold
        val_indices = list(zip(Y.nonzero()[0], Y.nonzero()[1]))
        val_values = Y.data

        # Compute the error only on the observed entries in the validation fold
        error = self._error_calc_validation(self.X, val_indices, val_values)
        return -error  # For scikit-learn, higher score is better

    def _error_calc(self, X_S, Y_S):

        # Convert sparse matrices to dense arrays if necessary
        if issparse(X_S):
            X_S = X_S.toarray()
        if issparse(Y_S):
            Y_S = Y_S.toarray()
        
        #Norm calculation X_S - Y_S
        normXminusY = np.linalg.norm(X_S - Y_S, ord=2)

        #Norm of Y_S
        norm_Y = np.linalg.norm(Y_S, ord=2)

        if norm_Y == 0:
            raise ValueError("Norm of Y is 0")

        #Calculate the error
        error = normXminusY/norm_Y
    
        return error

    def _error_calc_validation(self, X, val_indices, val_values):
        
        # Extract predicted values at the validation indices
        pred_values = np.array([X[i, j] for (i, j) in val_indices])

        # Compute the Frobenius norm of the difference
        numerator = np.linalg.norm(pred_values - val_values, ord=2)

        # Compute the Frobenius norm of the validation values
        denominator = np.linalg.norm(val_values, ord=2)

        if denominator == 0:
            raise ValueError("Norm of validation values is 0")

        # Compute the relative error
        error = numerator / denominator

        return error


### Selection of hyper parameter $\lambda$

It is very important to select the correct values for the hyper parameter lambda.
* $\lambda$ determines the threshold for the singular values
* Larger $\lambda$ values shrink more singular values towards zero, resulting in lower-rank matrices (under-fitting)
* Smaller $\lambda$ values do the opposite, resulting in better fit for the training data (over-fitting)
* Using the right $\lambda$ we can control the over/under fitting of the data

In [18]:
from sklearn.model_selection import KFold

# Fit once for timing purposes
svt_estimator = SVTEstimator(lambda_=10, learning_rate=1, max_iter=100, tol=1e-3)
svt_estimator.fit(sparse_matrix_train)
SVTTimingOneIter = svt_estimator.fit_time/svt_estimator.iter
print("Fit time SVT One iteration: ", SVTTimingOneIter)

# Define a range for lambda
lambda_values = np.logspace(-3, 1, 5)

# Perform cross-validation
cv_errors = []
for lambda_ in lambda_values:
    svt_estimator = SVTEstimator(lambda_=lambda_, learning_rate=1, max_iter=100, tol=1e-3)
    cv_scores = cross_val_score(svt_estimator, sparse_matrix_train, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=10)
    cv_errors.append(-np.mean(cv_scores))  # Convert scores to errors
    print("Cross validation errors: ", -cv_scores)
    print("Mean CV error: ", -np.mean(cv_scores))

# Find the best lambda
best_lambda = lambda_values[np.argmin(cv_errors)]
print(f"Best lambda: {best_lambda}")

# Best error
best_error_svt = np.min(cv_errors)
print(f"Best error: {best_error_svt}")



Fit time: 37.996586 seconds with 3 iterations and error 0.018712.
Fit time SVT One iteration:  12.665528535842896
Fit time: 18.509658 seconds with 2 iterations and error 0.008035.
Fit time: 18.689390 seconds with 2 iterations and error 0.008388.
Fit time: 18.958544 seconds with 2 iterations and error 0.007583.
Fit time: 24.393835 seconds with 3 iterations and error 0.007786.
Fit time: 26.080861 seconds with 3 iterations and error 0.007186.
Cross validation errors:  [0.96113299 0.96012607 0.937755   0.95479565 0.95684214]
Mean CV error:  0.9541303688695374
Fit time: 17.167175 seconds with 2 iterations and error 0.007786.
Fit time: 18.520374 seconds with 2 iterations and error 0.007186.Fit time: 18.501004 seconds with 2 iterations and error 0.008388.

Fit time: 32.532050 seconds with 4 iterations and error 0.008035.
Fit time: 33.822809 seconds with 4 iterations and error 0.007583.
Cross validation errors:  [0.96112345 0.96011541 0.93774133 0.95478493 0.95683224]
Mean CV error:  0.9541194

### Conclusion SVT
* SVT provides soft thresholding to the singular values
* This in essence is the nuclear norm regularization
* It is more robust to noise
* Convergense speed is much slower
* Useful when data is noisy
* Lambda parameter tuning is very important

## ADMiRA (Atomic decomposition for Minimization Risk Approximation)
ADMIRA is based on the idea of decomposing a signal or function into a combination of simpler, "atomic" components. These atoms are selected from a predefined dictionary (a set of basis functions or features) to best approximate the target signal. The goal is to achieve a sparse representation (using as few atoms as possible) while minimizing the approximation error or risk.

ADMIRA is an extension of the compressed sensing matching pursuit algorithm, where it iteratively selects "atoms" (rank 1 matrices) to approximate the target matrix, making it particularly useful for matrix completion and robust PCA problems. The approach is a "greedy".

The algorithm flow is as follows:  
Input the target rank.
* 1. SVD the residual and select the top 2 x Rank atoms (rank 1 matrices)
* 2. Add the selected atoms to the current approximation
* 3. Truncate to retain the top rank atoms
* 4. Update the residual (Y - X)
* 5. Calculate the error

In [19]:
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix, issparse
from scipy.sparse.linalg import svds
import time
from sklearn.base import BaseEstimator

class ADMIRAEstimator(BaseEstimator):
    def __init__(self, rank=10, max_iter=100, tol=1e-3):
        self.rank = rank
        self.max_iter = max_iter
        self.tol = tol
        self.X = None
        self.fit_time = None
        self.error = None
        self.iter = None

    def fit(self, Y, y=None):
        start_time = time.time()
        # Initialize the approximation matrix X
        self.X = lil_matrix(Y.shape, dtype=np.float64)
        residual = Y - self.X  # Initial residual
        iter = 0
        error = self.tol + 99
        prev_error = 100

        while error > self.tol and iter < self.max_iter and prev_error > error:
            # Update the previous error
            prev_error = error
            
            # Step 1: Select top 2 * rank atoms from the residual
            U, S, Vt = svds(residual, k=2 * self.rank)
            U = csr_matrix(U)
            Vt = csr_matrix(Vt)
            S = csr_matrix(np.diag(S))

            # Step 2: Combine the selected atoms with the current approximation
            X_candidate = self.X + U @ S @ Vt

            # Step 3: Truncate to retain only the top rank atoms
            U_trunc, S_trunc, Vt_trunc = svds(X_candidate, k=self.rank)
            U_trunc = csr_matrix(U_trunc)
            Vt_trunc = csr_matrix(Vt_trunc)
            S_trunc = csr_matrix(np.diag(S_trunc))
            self.X = U_trunc @ S_trunc @ Vt_trunc

            # Step 4: Update the residual
            residual = Y - self.X

            # Step 5: Calculate the error
            error = self._error_calc(residual, Y)
            iter += 1

        self.error = error
        self.iter = iter
        self.fit_time = time.time() - start_time
        print(f"Fit time: {self.fit_time:.6f} seconds with {self.iter} iterations and error {self.error:.6f}.")
        return self

    def score(self, Y, y=None):
        # Extract observed entries in the validation fold
        val_indices = list(zip(Y.nonzero()[0], Y.nonzero()[1]))
        val_values = Y.data

        # Compute the error only on the observed entries in the validation fold
        error = self._error_calc_validation(self.X, val_indices, val_values)
        return -error  # For scikit-learn, higher score is better

    def _error_calc(self, residual, Y):
        # Convert sparse matrices to dense arrays if necessary
        if issparse(residual):
            residual = residual.toarray()
        if issparse(Y):
            Y = Y.toarray()

        # Norm calculation residual
        norm_residual = np.linalg.norm(residual, ord=2)

        # Norm of Y
        norm_Y = np.linalg.norm(Y, ord=2)

        if norm_Y == 0:
            raise ValueError("Norm of Y is 0")

        # Calculate the error
        error = norm_residual / norm_Y

        return error

    def _error_calc_validation(self, X, val_indices, val_values):
        # Extract predicted values at the validation indices
        pred_values = np.array([X[i, j] for (i, j) in val_indices])

        # Compute the Frobenius norm of the difference
        numerator = np.linalg.norm(pred_values - val_values, ord=2)

        # Compute the Frobenius norm of the validation values
        denominator = np.linalg.norm(val_values, ord=2)

        if denominator == 0:
            raise ValueError("Norm of validation values is 0")

        # Compute the relative error
        error = numerator / denominator

        return error

In [21]:
admira = ADMIRAEstimator(rank=rank, max_iter=100, tol=1e-3)

#fit the model once for timing purposes
admira.fit(sparse_matrix_train)
ADMIRATimingOneIter = admira.fit_time/admira.iter
print("Fit time ADMIRA One iteration: ", ADMIRATimingOneIter)

#Perform 5 fold CV
cv_scores = cross_val_score(admira, sparse_matrix_train, cv=5, n_jobs=10)

print("Cross validation errors: ", -cv_scores)
print("Mean CV error: ", -np.mean(cv_scores))

Fit time: 127.031344 seconds with 4 iterations and error 0.058972.
Fit time ADMIRA One iteration:  31.757836043834686
Fit time: 61.415307 seconds with 2 iterations and error 0.052819.
Fit time: 63.879131 seconds with 2 iterations and error 0.055145.
Fit time: 85.789806 seconds with 3 iterations and error 0.057840.
Fit time: 86.200922 seconds with 3 iterations and error 0.055710.
Fit time: 90.464997 seconds with 3 iterations and error 0.054237.
Cross validation errors:  [0.95114829 0.9419772  0.95355127 0.95305303 0.95217684]
Mean CV error:  0.950381327654785
